In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import s2cell
from math import cos, asin, sqrt
import datetime as dt

In [2]:
#Check train test set
train = pd.read_csv('D:\\Downloads\\danthon\\data_train.csv')
train['train_test'] = 'train'
test = pd.read_csv('D:\\Downloads\\danthon\\data_test.csv')
test['train_test'] = 'test'

traintest_raw = pd.concat([train[['Ids','train_test','Labels']],test[['Ids','train_test']]]).reset_index(drop=True)
# cek_traintest = cek_traintest.merge(cekAlert,how='left',on='Ids').merge(cekIrreg,how='left',on='Ids').merge(cekIrregDec,how='left',on='Ids')

In [3]:
####################################### GET BASIC INFO for traintest_df

data = traintest_raw.copy()

########### Add additional info
data['date'] = pd.to_datetime(data['Ids'].apply(lambda x: x.split('_')[1]))
data['time'] = data['Ids'].apply(lambda x: int(x.split('_')[2]))

data['daytime'] = np.where((data['time'] >= 5) & (data['time'] < 12), 'morning', 
                         np.where((data['time'] >= 12) & (data['time'] < 16), 'noon',
                                 np.where((data['time'] >= 16) & (data['time'] < 23), 'evening',
                                         'midnight')))
data['weekday'] = data['date'].apply(lambda x: dt.datetime.strftime(x, '%A')[:3])

data['weekofmonth'] = data['date'].dt.day.apply(lambda x: 'w_'+str((x-1)//7+1))
data['date'].dt.day.apply(lambda x: 'w_'+str((x-1)//7+1))

data['isweekend'] = data['date'].dt.weekday.apply(lambda x: 1 if x>4 else 0)

data['isrushhour'] = np.where(((data['time'] >=6) & (data['time'] < 11)) |
                              ((data['time'] >=16) & (data['time'] < 20)), 1,0)

data['ispayday'] = np.where((data['date'].dt.day >= 24) | (data['date'].dt.day < 8), 1,0) 

# data['isholiday'] = np.where(data.isin({'date' : [dt.datetime(2020,10,28),dt.datetime(2020,10,29),
#                                                   dt.datetime(2020,10,30)]}),1,0)

data['isholiday'] = np.where(np.isin(data['date'],[dt.datetime(2020,10,28),dt.datetime(2020,10,29),
                                                  dt.datetime(2020,10,30)]),1,0)

data['s2token_15'] = data['Ids'].apply(lambda x: x.split('_')[0])


# result
data.head()

,Ids,train_test,Labels,date,time,daytime,weekday,weekofmonth,isweekend,isrushhour,ispayday,isholiday,s2token_15
0,2e69e9384_2020-10-06_13,train,True,2020-10-06,13,noon,Tue,w_1,0,0,1,0,2e69e9384
1,2e6992c7c_2020-10-02_17,train,True,2020-10-02,17,evening,Fri,w_1,0,1,1,0,2e6992c7c
2,2e69ef474_2020-09-13_19,train,True,2020-09-13,19,evening,Sun,w_2,1,1,0,0,2e69ef474
3,2e69c5fd4_2020-10-10_15,train,True,2020-10-10,15,noon,Sat,w_2,1,0,0,0,2e69c5fd4
4,2e6992134_2020-09-12_11,train,True,2020-09-12,11,morning,Sat,w_2,1,0,0,0,2e6992134


### Get Location information from Irreg and Alert data

In [4]:
#Get data from alert
dfAlertraw = pd.read_csv('D:\\Downloads\\danthon\\alerts.csv')

dfAlertraw['datetime'] = pd.to_datetime(dfAlertraw['pub_millis'],unit='ms')
dfAlertraw['datetime'] = dfAlertraw['datetime'] + pd.DateOffset(hours=7)
dfAlertraw['time'] = dfAlertraw['datetime'].dt.hour
dfAlertraw['daytime'] = np.where((dfAlertraw['time'] >= 5) & (dfAlertraw['time'] < 12), 'morning', 
                         np.where((dfAlertraw['time'] >= 12) & (dfAlertraw['time'] < 16), 'noon',
                                 np.where((dfAlertraw['time'] >= 16) & (dfAlertraw['time'] < 23), 'evening',
                                         'midnight')))
# dfAlertraw['isweekend'] = dfAlertraw['datetime'].dt.weekday.apply(lambda x: 1 if x>4 else 0)
dfAlertraw['weekday'] = dfAlertraw['datetime'].apply(lambda x: dt.datetime.strftime(x, '%A')[:3])

dfalerttype = dfAlertraw.groupby(['s2token_15','weekday','daytime'])['type','subtype'].apply(pd.DataFrame.mode).reset_index()

dfalertconf = dfAlertraw.groupby(['s2token_15','weekday','daytime'])['reliability','confidence'].mean().reset_index()
dfalertconf['flag_confidence'] = np.where((dfalertconf['reliability'] != 5)&(dfalertconf['confidence'] != 0) ,1,0)

dfalertroad = dfAlertraw.groupby('s2token_15')['road_type'].agg(lambda x:x.value_counts().index[0]).reset_index()    





dfAlert = dfAlertraw[['s2token_15','city']].drop_duplicates()
dfAlert = pd.merge(dfAlert,
                   pd.merge(dfalertroad,
                            pd.merge(dfalerttype,dfalertconf, how='left',on=['s2token_15','weekday','daytime'])
                            ,how='left',on='s2token_15')
                   ,how='left',on='s2token_15')
dfAlert = dfAlert[['s2token_15','city','weekday','daytime','type','subtype','road_type','flag_confidence']]
dfAlert.columns = ['s2token_15','city','weekday','daytime','ale_type','ale_subtype','ale_road_type',
                   'ale_flag_confidence']
dfAlert.head()

C:\Users\muhammad.aprilian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
C:\Users\muhammad.aprilian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app.launch_new_instance()


,s2token_15,city,weekday,daytime,ale_type,ale_subtype,ale_road_type,ale_flag_confidence
0,2e69eeea4,Depok,Fri,evening,JAM,JAM_STAND_STILL_TRAFFIC,1,1
1,2e69eeea4,Depok,Mon,evening,JAM,JAM_MODERATE_TRAFFIC,1,0
2,2e69eeea4,Depok,Sat,evening,JAM,JAM_STAND_STILL_TRAFFIC,1,1
3,2e69eeea4,Depok,Sat,morning,JAM,JAM_STAND_STILL_TRAFFIC,1,0
4,2e69eeea4,Depok,Sat,noon,JAM,JAM_STAND_STILL_TRAFFIC,1,0


In [5]:
print(dfAlertraw.shape)
print(dfAlert.shape)

(7800662, 22)
(21688, 8)


In [6]:
dfalerttype.shape

(21285, 6)

In [7]:
dfAlertraw[['s2token_15','city']].drop_duplicates().shape

(4385, 2)

In [8]:
dfalertroad.shape

(4341, 2)

In [9]:
dfAlert.isnull().sum()

s2token_15                0
city                      0
weekday                   0
daytime                   0
ale_type                152
ale_subtype            1838
ale_road_type             0
ale_flag_confidence       0
dtype: int64

In [10]:
#Get data from irreg
dfIrregraw = pd.read_csv('D:\\Downloads\\danthon\\irregularities.csv')
dfIrregraw = dfIrregraw.rename(columns={'s2token_center':'s2token_15'})

dfIrregraw['datetime'] = pd.to_datetime(dfIrregraw['update_date_millis'],unit='ms')
dfIrregraw['datetime'] = dfIrregraw['datetime'] + pd.DateOffset(hours=7)
dfIrregraw['time'] = dfIrregraw['datetime'].dt.hour
dfIrregraw['daytime'] = np.where((dfIrregraw['time'] >= 5) & (dfIrregraw['time'] < 12), 'morning', 
                         np.where((dfIrregraw['time'] >= 12) & (dfIrregraw['time'] < 16), 'noon',
                                 np.where((dfIrregraw['time'] >= 16) & (dfIrregraw['time'] < 23), 'evening',
                                         'midnight')))
# dfIrregraw['isweekend'] = dfIrregraw['datetime'].dt.weekday.apply(lambda x: 1 if x>4 else 0)
dfIrregraw['weekday'] = dfIrregraw['datetime'].apply(lambda x: dt.datetime.strftime(x, '%A')[:3])

dfIrregraw['type_score'] = np.where(dfIrregraw['type'] == 'Small',1,
                                    np.where(dfIrregraw['type'] == 'Medium',2,
                                            np.where(dfIrregraw['type'] == 'Large',3,4)))

dfIrrmean = dfIrregraw.groupby(['s2token_15','weekday','daytime'])['type_score','jam_level'].mean().reset_index()                                     
dfIrrmean = dfIrrmean[['s2token_15','weekday','daytime','type_score','jam_level']]
dfIrrmean.columns = ['s2token_15','weekday','daytime','irr_typescore_mean','irr_jamlevel_mean']

dfIrrmode = dfIrregraw.groupby(['s2token_15','weekday','daytime'])['type_score','jam_level'].agg(lambda x:x.value_counts().index[0]).reset_index()    
dfIrrmode = dfIrrmode[['s2token_15','weekday','daytime','type_score','jam_level']]
dfIrrmode.columns = ['s2token_15','weekday','daytime','irr_typescore_mode','irr_jamlevel_mode']


dfIrrsumm = dfIrregraw.groupby(['s2token_15','weekday','daytime']).count()['id'].reset_index()                                                                              
dfIrrsumm = dfIrrsumm[['s2token_15','weekday','daytime','id']]
dfIrrsumm.columns = ['s2token_15','weekday','daytime','irr_cnt']




dfIrreg = dfIrregraw[['s2token_15','city']].drop_duplicates()
dfIrreg = pd.merge(dfIrreg,
                   pd.merge(dfIrrsumm,
                            pd.merge(dfIrrmean,dfIrrmode, how='outer',on=['s2token_15','weekday','daytime'])
                            ,how='outer',on=['s2token_15','weekday','daytime'])
                  ,how='left',on='s2token_15')
dfIrreg = dfIrreg[['s2token_15','city','weekday','daytime',
                   'irr_cnt','irr_typescore_mean','irr_jamlevel_mean','irr_typescore_mode','irr_jamlevel_mode']]
dfIrreg.head()

C:\Users\muhammad.aprilian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\muhammad.aprilian\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,s2token_15,city,weekday,daytime,irr_cnt,irr_typescore_mean,irr_jamlevel_mean,irr_typescore_mode,irr_jamlevel_mode
0,2e69f2d2c,Bekasi,Fri,evening,352,1.281250,3.806818,1,4
1,2e69f2d2c,Bekasi,Fri,noon,59,1.525424,3.457627,1,3
2,2e69f2d2c,Bekasi,Mon,evening,100,1.450000,3.760000,1,4
3,2e69f2d2c,Bekasi,Mon,noon,75,1.253333,3.253333,1,3
4,2e69f2d2c,Bekasi,Sat,evening,376,1.210106,3.851064,1,4


In [11]:
print(dfIrregraw[['s2token_15','city']].drop_duplicates().shape)
print(dfIrreg.shape)

(1917, 2)
(9606, 9)


### Get combined data

In [12]:
dfToken = pd.merge(dfAlert,dfIrreg, how='outer', on=['s2token_15','city','weekday','daytime'])
dfToken = dfToken.reset_index(drop=True)
dfToken.head()

,s2token_15,city,weekday,daytime,ale_type,ale_subtype,ale_road_type,ale_flag_confidence,irr_cnt,irr_typescore_mean,irr_jamlevel_mean,irr_typescore_mode,irr_jamlevel_mode
0,2e69eeea4,Depok,Fri,evening,JAM,JAM_STAND_STILL_TRAFFIC,1.0,1.0,20.0,1.800000,4.000000,2.0,4.0
1,2e69eeea4,Depok,Mon,evening,JAM,JAM_MODERATE_TRAFFIC,1.0,0.0,31.0,1.129032,2.870968,1.0,3.0
2,2e69eeea4,Depok,Sat,evening,JAM,JAM_STAND_STILL_TRAFFIC,1.0,1.0,18.0,1.000000,4.000000,1.0,4.0
3,2e69eeea4,Depok,Sat,morning,JAM,JAM_STAND_STILL_TRAFFIC,1.0,0.0,38.0,1.552632,4.000000,2.0,4.0
4,2e69eeea4,Depok,Sat,noon,JAM,JAM_STAND_STILL_TRAFFIC,1.0,0.0,89.0,1.033708,3.955056,1.0,4.0


In [13]:
dfToken.shape

(25358, 13)

In [14]:
#Get Latitude and Longitude from s2token 15
dfToken['latlon'] = dfToken['s2token_15'].apply(lambda x: s2cell.token_to_lat_lon(x))
dfToken['lat'] = dfToken['latlon'].apply(lambda x: x[0])
dfToken['lon'] = dfToken['latlon'].apply(lambda x: x[1])
dfToken['latlon_dict'] = dfToken[['lat','lon']].apply(lambda x: x.to_dict(),axis=1)

#getting predictor
col_token = list(dfToken.iloc[:,[4,5,6,7,8,9,10,11,12]])

################# GET LAT LON LIST FOR EACH CITY - STREET COMBINATION
# Assume the center of the point by averaging the lat lon
# dfCity = dfToken.groupby(['city','street']).agg({'lat':['mean'],'lon':['mean']}).reset_index()
# dfCity.columns = ['street','city','lat','lon']
# dfAdd = dfToken[['street','city','road_type','is_highway','lat','lon']].drop_duplicates().reset_index(drop=True)
# dfAdd['latlon'] = dfAdd[['lat', 'lon']].apply(tuple, axis=1)
# dfAdd['latlon_dict'] = dfAdd[['lat','lon']].apply(lambda x: x.to_dict(),axis=1)


#INPUT FOR get_city FUNCTION
listToken = list(dfToken['latlon_dict'])

In [15]:
col_token

['ale_type',
 'ale_subtype',
 'ale_road_type',
 'ale_flag_confidence',
 'irr_cnt',
 'irr_typescore_mean',
 'irr_jamlevel_mean',
 'irr_typescore_mode',
 'irr_jamlevel_mode']

In [16]:
################## GET CLOSEST LOCATION FROM LIST OF LOCATIONS AND POINTS
#SOURCE https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

def get_city(x):
    temp_closest= closest(listToken, x)
    
    cek = dfToken.loc[(dfToken.lat==temp_closest['lat'])&(dfToken.lon==temp_closest['lon'])].reset_index(drop=True)
#     street = cek.loc[0,'street']
    city = cek.loc[0,'city']
    return city

def get_all(x):
    temp_closest= closest(listToken, x['latlondict'])
    
    mask_city = (dfToken.lat==temp_closest['lat'])&(dfToken.lon==temp_closest['lon']) 
    mask_wday = (dfToken.weekday==x['weekday'])
    mask_dtime = (dfToken.daytime==x['daytime'])

    cek = dfToken[mask_city&mask_wday&mask_dtime]
    if cek.empty:
        pd.DataFrame(np.nan,columns=col_token,index=[0]).reset_index(drop=True)
    else: 
        return cek[col_token].iloc[0]

In [17]:
# Gather unique s2token to 1 df then merge it back to the main df for more speed
token_temp = data[['s2token_15','weekday','daytime']].copy().drop_duplicates().reset_index(drop=True)

#Get Latitude and Longitude from s2token 15
token_temp['latlon'] = token_temp['s2token_15'].apply(lambda x: s2cell.token_to_lat_lon(x))
token_temp['lat'] = token_temp['latlon'].apply(lambda x: x[0])
token_temp['lon'] = token_temp['latlon'].apply(lambda x: x[1])
token_temp['latlondict'] = token_temp[['lat','lon']].apply(lambda x: x.to_dict(),axis=1)

# Get the new city,street,road_type,and highway
token_temp['city_pred'] = token_temp['latlondict'].apply(lambda x: get_city(x))
# token_temp['city_pred'] = token_temp['location_pred'].apply(lambda x: x[1])

token_temp[col_token] = token_temp.apply(lambda x: get_all(x),axis=1)

In [18]:
#merge back to main df
data = data.merge(token_temp,how='left',on=['s2token_15','weekday','daytime'])

In [19]:
data.shape

(85177, 27)

In [20]:
data.isnull().sum()

Ids                        0
train_test                 0
Labels                 13841
date                       0
time                       0
daytime                    0
weekday                    0
weekofmonth                0
isweekend                  0
isrushhour                 0
ispayday                   0
isholiday                  0
s2token_15                 0
latlon                     0
lat                        0
lon                        0
latlondict                 0
city_pred                  0
ale_type               13932
ale_subtype            16248
ale_road_type          13932
ale_flag_confidence    13932
irr_cnt                37980
irr_typescore_mean     37980
irr_jamlevel_mean      37980
irr_typescore_mode     37980
irr_jamlevel_mode      37980
dtype: int64

In [27]:
data[['Ids','train_test','daytime','weekday','weekofmonth','isweekend','isrushhour','ispayday', 'isholiday',
      'city_pred','ale_type','ale_subtype','ale_road_type','irr_cnt','irr_typescore_mean',
      'irr_jamlevel_mean','irr_typescore_mode','irr_jamlevel_mode']].to_csv('traintest_base_wc.csv',index=False)

In [ ]:
data[['Ids','train_test','daytime','weekday','weekofmonth','isweekend','isrushhour','ispayday',
              'city_pred','street_pred','roadtype_pred','highway_pred']]

In [ ]:
data['highway_pred'].loc[traintest_df['train_test']=='test'].value_counts()

In [ ]:
data['roadtype_pred'].loc[traintest_df['train_test']=='test'].value_counts()

In [22]:
data['ale_type'] = np.where(data['ale_flag_confidence'] == 1,data['ale_type'],np.nan)
data['ale_subtype'] = np.where(data['ale_flag_confidence'] == 1,data['ale_subtype'],np.nan)
data['ale_road_type'] = np.where(data['ale_flag_confidence'] == 1,data['ale_road_type'],np.nan)


In [23]:
data

,Ids,train_test,Labels,date,time,daytime,weekday,weekofmonth,isweekend,isrushhour,...,city_pred,ale_type,ale_subtype,ale_road_type,ale_flag_confidence,irr_cnt,irr_typescore_mean,irr_jamlevel_mean,irr_typescore_mode,irr_jamlevel_mode
0,2e69e9384_2020-10-06_13,train,True,2020-10-06,13,noon,Tue,w_1,0,0,...,Depok,NaN,NaN,NaN,0.0,6.0,1.000000,2.333333,1.0,2.0
1,2e6992c7c_2020-10-02_17,train,True,2020-10-02,17,evening,Fri,w_1,0,1,...,Bekasi,NaN,NaN,NaN,NaN,19.0,1.684211,3.000000,2.0,3.0
2,2e69ef474_2020-09-13_19,train,True,2020-09-13,19,evening,Sun,w_2,1,1,...,Depok,NaN,NaN,NaN,0.0,62.0,1.338710,3.661290,1.0,4.0
3,2e69c5fd4_2020-10-10_15,train,True,2020-10-10,15,noon,Sat,w_2,1,0,...,Bogor,JAM,JAM_STAND_STILL_TRAFFIC,6.0,1.0,2.0,1.000000,4.000000,1.0,4.0
4,2e6992134_2020-09-12_11,train,True,2020-09-12,11,morning,Sat,w_2,1,0,...,Bekasi,JAM,JAM_STAND_STILL_TRAFFIC,7.0,1.0,42.0,1.452381,3.309524,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85172,2e68dd414_2020-11-26_5,test,NaN,2020-11-26,5,morning,Thu,w_4,0,0,...,Bandung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85173,2e698541c_2020-11-24_22,test,NaN,2020-11-24,22,evening,Tue,w_4,0,0,...,Cikarang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85174,2e69e8e0c_2020-11-24_10,test,NaN,2020-11-24,10,morning,Tue,w_4,0,1,...,Depok,NaN,NaN,NaN,0.0,64.0,1.796875,3.578125,1.0,4.0
85175,2e699a1cc_2020-11-24_18,test,NaN,2020-11-24,18,evening,Tue,w_4,0,1,...,Cikarang,NaN,NaN,NaN,0.0,9.0,1.000000,3.000000,1.0,3.0


In [24]:
data.isnull().sum()

Ids                        0
train_test                 0
Labels                 13841
date                       0
time                       0
daytime                    0
weekday                    0
weekofmonth                0
isweekend                  0
isrushhour                 0
ispayday                   0
isholiday                  0
s2token_15                 0
latlon                     0
lat                        0
lon                        0
latlondict                 0
city_pred                  0
ale_type               63999
ale_subtype            64066
ale_road_type          63999
ale_flag_confidence    13932
irr_cnt                37980
irr_typescore_mean     37980
irr_jamlevel_mean      37980
irr_typescore_mode     37980
irr_jamlevel_mode      37980
dtype: int64